# Synergic quantum generative machine learning- single qubit case

A program that trains circuits with a given number of qubits on a quantum computer for SQGEN approache suggested in the paper: "Synergic quantum generative machine learning" (arXiv:2112.13255v2)

Import IBMQ account and necessary libraries

In [ ]:
#MY_API_TOKEN=""
#from qiskit import IBMQ
#IBMQ.save_account(MY_API_TOKEN, overwrite = True)
#IBMQ.load_account()
#backend = provider.get_backend('ibmq_montreal')

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
from qiskit.test.mock.backends import FakeMontreal
backend = FakeMontreal()


In [ ]:
from numpy import pi
import numpy as np
from qiskit import QuantumCircuit, execute, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.extensions.unitary import UnitaryGate
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import Aer, assemble, IBMQ

# Get the noise model
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map
from qiskit.circuit.library import MCMT, RZGate, RYGate

# Get basis gates from noise model
basis_gates = noise_model.basis_gates
from qiskit.extensions import Initialize
from numpy import sqrt 

Circuit corresponding to the trained generator

In [ ]:
def GeneratorG(n,xG):
    x = ParameterVector('x',4*n)
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr, name='G')
    for i in range(n):
        if i < n-1:
            gate = MCMT(RZGate(2*pi*x[i]),n-i-1, 1)
            qc.append(gate,[m for m in range(n-i)])
        else:
            qc.rz(2*pi*x[i],0)
    for i in range(n):
        if i < n-1:
            gate = MCMT(RYGate(2*pi*x[i+n]),n-i-1, 1)
            qc.append(gate,[m for m in range(n-i)])
        else:
            qc.ry(2*pi*x[i+n],0)
    for i in reversed(range(n)):
        if i < n-1:
            gate = MCMT(RYGate(2*pi*x[i+3*n]),n-i-1, 1)
            qc.append(gate,[m for m in range(n-i)])
        else:
            qc.ry(2*pi*x[i+3*n],0)
    for i in reversed(range(n)):
        if i < n-1:
            gate = MCMT(RZGate(2*pi*x[i+2*n]),n-i-1, 1)
            qc.append(gate,[m for m in range(n-i)])
        else:
            qc.rz(2*pi*x[i+2*n],0)
    result = qc.to_instruction({x:xG})
    return result


Circuit corresponding to the real data generator and its conjugate transposition

In [ ]:
def GeneratorR(n,xR):
    x =Parameter('x')
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr, name='R')
    v = [1/sqrt(2)]+(2**n-2)*[0] + [1/sqrt(2)] #GHZ state
    qc.initialize(v,[m for m in range(n)])
    result = qc.to_instruction()
    return result
  
def GeneratorRdg(n,xR):
    x =Parameter('x')
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr, name='Rdg')
    v = [1/sqrt(2)]+(2**n-2)*[0] + [1/sqrt(2)] #GHZ state    
    qi=Initialize(v).gates_to_uncompute()
    qi = qi.to_instruction()
    qc.append(qi,qr)
    #result = qc.to_instruction({x:xR})
    result = qc.to_instruction()
    return result

Circuit corresponding to the trained discriminator

In [ ]:
def Discriminator(n,xD,testing=False):
    qr = QuantumRegister(n+1)
    qc = QuantumCircuit(qr, name='D')
    
    xD1=ParameterVector('xD1',4*n)
    sub_inst= GeneratorG(n,xD1)
    qc.append(sub_inst, [qr[i] for i in range(n)])
    
    shift = pi/2
    if testing:
        qc.cry(shift,qr[n-1],qr[n])
    else:
        qc.cry(shift/2,qr[n-1],qr[n])
    
    qc.append(sub_inst.inverse(), [qr[i] for i in range(n)])

    result = qc.to_instruction({xD1:xD[0:4*n]
                               })
    
    return result

In [ ]:
n = 1 #number of qubits
myshots = 32000 #number of shots

Circuits responsible for the learning process

In [ ]:
xG = ParameterVector('xG',4*n)
xD = ParameterVector('xD',4*n)  
xR = Parameter('xR')
D = Discriminator(n,xD)
Dt = Discriminator(n,xD,testing=True)
G = GeneratorG(n,xG)
R = GeneratorR(n,xR)
Rdg = GeneratorRdg(n,xR)

qr = QuantumRegister(n+1, 'q')
cr = ClassicalRegister(n+1, 'c')
qc = QuantumCircuit(qr,cr)
qc.append(R,[qr[i] for i in range(n)])
qc.append(D,[qr[i] for i in range(n+1)])
qc.x(qr[n])
qc.append(D.inverse(),[qr[i] for i in range(n+1)])
qc.append(G.inverse(),[qr[i] for i in range(n)])
qc.measure(qr, cr)
q = transpile(qc,backend)


qrR = QuantumRegister(n+1,'q')
crR = ClassicalRegister(n+1, 'c')
qcR = QuantumCircuit(qrR,crR)
qcR.append(R,[qrR[i] for i in range(n)])
qcR.append(Dt,[qrR[i] for i in range(n+1)])
qcR.append(Rdg,[qrR[i] for i in range(n)])
qcR.measure(qrR, crR)
qR = transpile(qcR,backend)

qrG = QuantumRegister(n+1,'q')
crG = ClassicalRegister(n+1,'c')
qcG = QuantumCircuit(qrG,crG)
qcG.append(G,[qrG[i] for i in range(n)])
qcG.append(Dt,[qrG[i] for i in range(n+1)])
qcG.append(G.inverse(),[qrG[i] for i in range(n)])
qcG.measure(qrG, crG)
qG = transpile(qcG,backend)


qrRG = QuantumRegister(n,'q')
crRG = ClassicalRegister(n,'c')
qcRG = QuantumCircuit(qrRG,crRG)
qcRG.append(R,[qrRG[i] for i in range(n)])
qcRG.append(G.inverse(),[qrRG[i] for i in range(n)])
qcRG.measure(qrRG, crRG)
qRG = transpile(qcRG,backend)

Loops used to calculate the probabilities of correct recognition of states by the discriminator, and the fidelity of the obtained states

In [ ]:
def real_true(n,x,prt=[]):
    global qR,xD
    global myshots, coupling_map, basis_gates, backend    
    
    b = {xD:x[:4*n]} # ,xR:0
    
    
    qRb = qR.bind_parameters(b)
    job = execute(qRb, 
                 backend,
                 shots=myshots,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates)
    
    result = job.result()
    counts = result.get_counts()
    c00 = 0
    c10 = 0
    c01 = 0
    c11 = 0
    if ('00' in counts.keys()):
        c00 = float(counts['00'])
    if ('10' in counts.keys()):
        c10 = float(counts['10'])
    if ('01' in counts.keys()):
        c01 = float(counts['01'])
    if ('11' in counts.keys()):
        c11 = float(counts['11'])
    
    Sc = float(myshots)
    result = c00 /Sc    
    prt.append(result)
    return 1- result 

def fake_true(n,x,pft=[]):
    global qG,xD
    global myshots, coupling_map, basis_gates, backend
    b = {xD:x[:(4*n)],xG:x[(4*n):(4*n+4*n)]}
    qGb = qG.bind_parameters(b)
    
    job = execute(qGb, 
                 backend,
                 shots=myshots,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates)
    
    result = job.result()
    counts = result.get_counts()
    c00 = 0
    c10 = 0
    c01 = 0
    c11 = 0
    if ('00' in counts.keys()):
        c00 = float(counts['00'])
    if ('10' in counts.keys()):
        c10 = float(counts['10'])
    if ('01' in counts.keys()):
        c01 = float(counts['01'])
    if ('11' in counts.keys()):
        c11 = float(counts['11'])
    Sc = float(c00 + c01 + c10 + c11)
    result = c00/Sc    
    pft.append(result)
    return 1 - result
    
def fidelityRG(n,x,fid=[]):
    global qRG,xG
    global myshots, coupling_map, basis_gates, backend
    b = {xG:x[(4*n):(4*n+4*n)]} # xR:0,
    
    qRGb = qRG.bind_parameters(b)
    
    
    job = execute(qRGb,
                 backend,
                 shots=myshots,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates)
    
    
    result = job.result()
    counts = result.get_counts()
    c0 = 0
    c1 = 0
    if ('0' in counts.keys()):
        c0 = float(counts['0'])
    if ('1' in counts.keys()):
        c1 = float(counts['1'])
    Sc = float(myshots)
    result = c0/Sc    
    fid.append(result)
    return result

Loops to calculating the total cost function

In [ ]:
def costNew(n,x,verbose=False):
    global q,qc,xG,xD,costNew_evals, costNew_iter
    global prt_new, pft_new, fid_new
    global myshots, coupling_map, basis_gates, backend
    costNew_evals+=1
    
    b = {xD:x[:(4*n)],xG:x[(4*n):(4*n+4*n)]} # ,xR:0
    
    qb = q.bind_parameters(b)
    
    job = execute(qb,
                 backend,
                 shots=myshots,
                 coupling_map=coupling_map,
                 basis_gates=basis_gates)
    
    result = job.result()
    counts = result.get_counts()
    c00 = 0
    c10 = 0
    c01 = 0
    c11 = 0
    if ('00' in counts.keys()):
        c00 = float(counts['00'])
    if ('10' in counts.keys()):
        c10 = float(counts['10'])
    if ('01' in counts.keys()):
        c01 = float(counts['01'])
    if ('11' in counts.keys()):
        c11 = float(counts['11'])
    Sc = float(myshots)
    result = 1-c00 /Sc    
    if verbose: print(np.array([result,
                                real_true(n,x,prt_new),
                                fake_true(n,x,pft_new),
                                fidelityRG(n,x,fid_new)]),end="\r")
    return result 

Learning process according to the SQGEN approach

In [ ]:
itrNew = 15
seed = 42
from scipy.optimize import minimize

opt = {'maxiter': 6,
       'disp':True,
       'eps':1e-8,
       'finite_diff_rel_step':1e-8} 
alg = 'BFGS'#'CG'
alg = 'Nelder-Mead'


np.random.seed(seed)  
x0 = np.random.rand((4*n+4*n))

costNew_evals = 0
prt_new = []
pft_new = []
fid_new = []
cost_new = []
prt_iter_new = []
pft_iter_new = []
fid_iter_new = []
costNew_iter = []

for m in range(itrNew):
    fake_true(n,x0,pft_iter_new)  
    real_true(n,x0,prt_iter_new)  
    fidelityRG(n,x0,fid_iter_new)
    cost_new.append(costNew(n,x0))
    sol = minimize(lambda x: costNew(n,x,True),x0,method  = alg, options = opt) 
    x0 = sol.x

fake_true(n,x0,pft_iter_new)  
real_true(n,x0,prt_iter_new)  
fidelityRG(n,x0,fid_iter_new)
print(costNew_evals)  


Saving data to files

In [ ]:
np.save("prt_new.npy",prt_new)
np.save("pft_new.npy",pft_new)
np.save("fid_new.npy",fid_new)
np.save("prt_iter_new.npy",prt_iter_new)
np.save("pft_iter_new.npy",pft_iter_new)
np.save("fid_iter_new.npy",fid_iter_new)
np.save("cost_new.npy",cost_new)